In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow as tf
from keras.models import Model
from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.model import Sequential
from deel.lip.activations import GroupSort, FullSort
from Autoencoderclasses import Encoder, Decoder, DenseEncoder, DenseDecoder 
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from firstprinciples import diff_schemes
import matplotlib.pyplot as plt
from tensorflow.keras.losses import MeanSquaredError 
import pandas as pd

gamma = 4.0
beta_T = 50.0
beta_U = 2.0
Delta = 0.8
h_c = 0.002
L = np.pi
N_s = 45
N_integration = int(Delta/h_c)
delta_z = L/N_s
testsize = 200


np.random.seed(123)

from pickle import load
scaler_X = load(open('scaler_X.pkl', 'rb'))
scaler_y = load(open('scaler_y.pkl', 'rb'))




In [ ]:


title = 'Dense 300 units 6 rd 1.0 noise'

if title[0] == 'S':
    encoder_and_decoder = tf.keras.models.load_model(title + ".h5", custom_objects={'Encoder' : Encoder, 'Decoder' : Decoder})
    def test_model_output(initial_state,u1,u2):
        encoder = encoder_and_decoder.layers[0]
        decoder = encoder_and_decoder.layers[1]
        initial_state = initial_state.copy()
        initial_state_normalized = scaler_X.transform([initial_state])
        input = np.concatenate((encoder.call(np.array(initial_state_normalized).reshape((1,46))),np.array([u1/100.0,u2/100.0])), axis = None)
        input = np.tile(input, (10,1))
        RNN_model = tf.keras.models.load_model('RNN for ' + title + '.h5')
        RNN_output = RNN_model.predict(input.reshape((1,10,8)),verbose = 0)
        output = decoder.call(RNN_output)
        return np.vstack((initial_state,scaler_X.inverse_transform(np.array(output))))
else:
    encoder_and_decoder = tf.keras.models.load_model(title + ".h5", custom_objects={'DenseEncoder' : DenseEncoder, 'DenseDecoder' : DenseDecoder})
    def test_model_output(initial_state,u1,u2):
        encoder = encoder_and_decoder.layers[0]
        decoder = encoder_and_decoder.layers[1]
        initial_state = initial_state.copy()
        initial_state_normalized = scaler_X.transform([initial_state])
        input = np.concatenate((encoder.call(np.array(initial_state_normalized).reshape((1,46))),np.array([u1/100.0,u2/100.0])), axis = None)
        input = np.tile(input, (10,1))
        RNN_model = tf.keras.models.load_model('RNN for ' + title + '.h5')
        RNN_output = RNN_model.predict(input.reshape((1,10,8)),verbose = 0)
        output = decoder.call(RNN_output)
        return np.vstack((initial_state,scaler_X.inverse_transform(np.array(output[0]))))



    




def firstprinciples_output(initial_state,u1,u2):
    dim_x = 45
    dim_t = 100
    dt = 0.2/dim_t 
    dx = np.pi/dim_x
    a = dt/dx**2
    X_array = np.linspace(0,np.pi,(dim_x+1))
    X_array = np.array(X_array)
    T_array = np.linspace(0,0.2,dim_t+1)
    res = diff_schemes(dt,dx,a,X_array,T_array,u1,u2,initial_state)
    rfd = res.forward_diff_scheme()
    output = rfd[::10]
    return np.vstack((initial_state,np.array(output[1::])))




y_predict = test_model_output(np.array([6]*46),-100,-100)
y_actual = firstprinciples_output(np.array([6]*46),-100,-100)

x = np.linspace(0.0,np.pi ,46  )
y = np.linspace(0, 0.2, 11)
X, Y = np.meshgrid(x, y)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X, Y, np.abs(y_predict-y_actual) , 50, cmap='plasma')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

mse = tf.keras.losses.MeanSquaredError()
error = mse(np.array(y_predict),np.array(y_actual)).numpy()
print("Mean Squared Error", error)

df_y_predict = pd.DataFrame(y_predict)
filepath = 'OpenLoop ' + title + '.xlsx'
df_y_predict.to_excel(filepath, index=False)

df_y_actual = pd.DataFrame(y_actual)
filepath = 'OpenLoopActual.xlsx'
df_y_actual.to_excel(filepath, index=False)


df_y_error = pd.DataFrame(np.abs(y_predict-y_actual))
filepath = 'OpenLoop ' + title + ' error.xlsx'
df_y_error.to_excel(filepath, index=False)
